In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
import time
warnings.filterwarnings('ignore')

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Paths
BASE_DIR = Path(r'c:\Users\Acer\Desktop\Forex-Signal-App')
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v14'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# GPU Check
import torch
GPU_AVAILABLE = torch.cuda.is_available()

# Trading Parameters
TP_PIPS = 15
SL_PIPS = 10
FORWARD_BARS = 60

print("="*70)
print("🚀 FOREX SIGNAL GENERATOR V14")
print("   V10 Features + TP/SL Based Labeling")
print("="*70)
print(f"✓ GPU Available: {GPU_AVAILABLE}")
print(f"✓ TP: {TP_PIPS} pips, SL: {SL_PIPS} pips")
print(f"✓ Forward window: {FORWARD_BARS} bars")
print(f"✓ Model Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL GENERATOR V14
   V10 Features + TP/SL Based Labeling
✓ GPU Available: True
✓ TP: 15 pips, SL: 10 pips
✓ Forward window: 60 bars
✓ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v14


## 1. Load Data

In [2]:
# Load Data
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

for df in [train_df, test_df]:
    if 'timestamp' in df.columns:
        df.rename(columns={'timestamp': 'time'}, inplace=True)
    df['time'] = pd.to_datetime(df['time'])

print(f"Train: {len(train_df):,} rows")
print(f"Test: {len(test_df):,} rows")
print(f"Train period: {train_df['time'].min()} to {train_df['time'].max()}")
print(f"Test period: {test_df['time'].min()} to {test_df['time'].max()}")

Train: 1,859,492 rows
Test: 296,778 rows
Train period: 2019-12-31 16:00:00+00:00 to 2024-12-30 16:00:00+00:00
Test period: 2024-12-31 16:00:00+00:00 to 2025-10-17 06:11:00+00:00


## 2. V10-Style Feature Engineering

In [3]:
def add_features_v14(df):
    """
    V14 Features: V10-ийн бүх features + V8 composite features
    """
    df = df.copy()
    
    # ==================== CORE FEATURES ====================
    # Time Features
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)
    
    # Moving Averages
    for p in [5, 10, 20, 50, 200]:
        df[f'sma_{p}'] = df['close'].rolling(p).mean()
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df['close'].ewm(span=12).mean()
    ema26 = df['close'].ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands
    df['bb_mid'] = df['close'].rolling(20).mean()
    df['bb_std'] = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_mid'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (df['bb_mid'] + 1e-10)
    
    # ATR & ADX
    df['tr0'] = abs(df['high'] - df['low'])
    df['tr1'] = abs(df['high'] - df['close'].shift())
    df['tr2'] = abs(df['low'] - df['close'].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
    
    df['up_move'] = df['high'] - df['high'].shift()
    df['down_move'] = df['low'].shift() - df['low']
    
    df['plus_dm'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
    df['minus_dm'] = np.where((df['down_move'] > df['up_move']) & (df['down_move'] > 0), df['down_move'], 0)
    
    period = 14
    df['atr'] = df['tr'].rolling(period).mean()
    df['plus_di'] = 100 * (df['plus_dm'].rolling(period).mean() / (df['atr'] + 1e-10))
    df['minus_di'] = 100 * (df['minus_dm'].rolling(period).mean() / (df['atr'] + 1e-10))
    df['dx'] = 100 * abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'] + 1e-10)
    df['adx'] = df['dx'].rolling(period).mean()
    
    # CCI
    tp = (df['high'] + df['low'] + df['close']) / 3
    sma_tp = tp.rolling(20).mean()
    mad_tp = tp.rolling(20).apply(lambda x: np.abs(x - x.mean()).mean())
    df['cci'] = (tp - sma_tp) / (0.015 * mad_tp + 1e-10)
    
    # Williams %R
    hh = df['high'].rolling(14).max()
    ll = df['low'].rolling(14).min()
    df['williams_r'] = -100 * (hh - df['close']) / (hh - ll + 1e-10)
    
    # Volatility
    df['returns'] = df['close'].pct_change()
    df['volatility'] = df['returns'].rolling(20).std() * 100
    
    # ==================== COMPOSITE FEATURES ====================
    df['rsi_x_adx'] = df['rsi'] * df['adx'] / 100
    df['momentum_score'] = (
        (df['rsi'] > 50).astype(int) + 
        (df['macd'] > df['macd_signal']).astype(int) + 
        (df['plus_di'] > df['minus_di']).astype(int)
    )
    df['price_position'] = (df['close'] - df['sma_50']) / (df['atr'] + 1e-10)
    df['trend_score'] = (
        (df['close'] > df['sma_20']).astype(int) +
        (df['sma_20'] > df['sma_50']).astype(int) +
        (df['sma_50'] > df['sma_200']).astype(int) +
        (df['adx'] > 25).astype(int)
    )
    df['rsi_zone'] = pd.cut(df['rsi'], bins=[0, 30, 45, 55, 70, 100], labels=[0, 1, 2, 3, 4]).astype(float)
    df['macd_momentum'] = df['macd_hist'] - df['macd_hist'].shift(3)
    df['close_vs_high'] = (df['high'].rolling(20).max() - df['close']) / (df['atr'] + 1e-10)
    df['close_vs_low'] = (df['close'] - df['low'].rolling(20).min()) / (df['atr'] + 1e-10)
    
    # ==================== V10 ENHANCED FEATURES ====================
    # Trend Strength Score (0-5)
    df['trend_strength'] = (
        (df['close'] > df['ema_5']).astype(int) +
        (df['ema_5'] > df['ema_10']).astype(int) +
        (df['ema_10'] > df['ema_20']).astype(int) +
        (df['ema_20'] > df['ema_50']).astype(int) +
        (df['adx'] > 20).astype(int)
    )
    
    # Momentum Alignment
    df['momentum_alignment'] = (
        (df['rsi'] > 55).astype(int) +
        (df['macd_hist'] > 0).astype(int) +
        (df['cci'] > 50).astype(int) +
        (df['williams_r'] > -30).astype(int)
    )
    
    # Volatility State
    df['volatility_sma'] = df['volatility'].rolling(50).mean()
    df['volatility_state'] = np.where(
        df['volatility'] > df['volatility_sma'] * 1.5, 2,
        np.where(df['volatility'] < df['volatility_sma'] * 0.5, 0, 1)
    )
    
    # Price Action Patterns
    df['body'] = df['close'] - df['open']
    df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
    df['body_ratio'] = abs(df['body']) / (df['high'] - df['low'] + 1e-10)
    
    # Bullish/Bearish patterns
    df['is_bullish'] = (df['close'] > df['open']).astype(int)
    df['bullish_streak'] = df['is_bullish'].rolling(5).sum()
    
    # Support/Resistance proximity
    df['dist_to_high20'] = (df['high'].rolling(20).max() - df['close']) / (df['atr'] + 1e-10)
    df['dist_to_low20'] = (df['close'] - df['low'].rolling(20).min()) / (df['atr'] + 1e-10)
    
    # Multi-timeframe RSI
    df['rsi_5'] = df['rsi'].rolling(5).mean()
    df['rsi_20'] = df['rsi'].rolling(20).mean()
    df['rsi_trend'] = df['rsi_5'] - df['rsi_20']
    
    # Breakout Detection
    df['above_bb_upper'] = (df['close'] > df['bb_upper']).astype(int)
    df['below_bb_lower'] = (df['close'] < df['bb_lower']).astype(int)
    df['bb_breakout'] = df['above_bb_upper'] - df['below_bb_lower']
    
    # Price Momentum
    df['price_change_5'] = (df['close'] - df['close'].shift(5)) / (df['atr'] + 1e-10)
    df['price_change_10'] = (df['close'] - df['close'].shift(10)) / (df['atr'] + 1e-10)
    df['price_change_20'] = (df['close'] - df['close'].shift(20)) / (df['atr'] + 1e-10)
    
    # Session Quality
    df['session_quality'] = df['is_london'].astype(int) + df['is_ny'].astype(int) + df['is_overlap'].astype(int) * 2
    
    # Cleanup temp columns
    drop_cols = ['tr0', 'tr1', 'tr2', 'tr', 'up_move', 'down_move', 'plus_dm', 'minus_dm']
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)
    
    return df

print("Adding V14 features (V10-style)...")
train_df = add_features_v14(train_df)
test_df = add_features_v14(test_df)
print(f"✓ Features added. Total columns: {len(train_df.columns)}")

Adding V14 features (V10-style)...
✓ Features added. Total columns: 69


## 3. TP/SL Based Labeling (BUY-only)

In [4]:
def create_tpsl_labels(df, tp_pips=15, sl_pips=10, forward_bars=60):
    """
    V14 TP/SL Based Labeling for BUY signals:
    - Label = 1 (WIN): TP hit before SL
    - Label = 0 (LOSE): SL hit before TP
    - Label = -1 (NO_TRADE): Neither hit within forward_bars
    """
    df = df.copy()
    tp_price = tp_pips * 0.0001
    sl_price = sl_pips * 0.0001
    
    labels = []
    
    for i in range(len(df)):
        if i + forward_bars >= len(df):
            labels.append(-1)
            continue
            
        entry_price = df.iloc[i]['close']
        tp_level = entry_price + tp_price
        sl_level = entry_price - sl_price
        
        future = df.iloc[i+1:i+1+forward_bars]
        
        tp_hit_idx = None
        sl_hit_idx = None
        
        tp_hits = future[future['high'] >= tp_level]
        if len(tp_hits) > 0:
            tp_hit_idx = tp_hits.index[0]
        
        sl_hits = future[future['low'] <= sl_level]
        if len(sl_hits) > 0:
            sl_hit_idx = sl_hits.index[0]
        
        if tp_hit_idx is not None and sl_hit_idx is not None:
            if tp_hit_idx < sl_hit_idx:
                labels.append(1)  # WIN
            else:
                labels.append(0)  # LOSE
        elif tp_hit_idx is not None:
            labels.append(1)  # WIN
        elif sl_hit_idx is not None:
            labels.append(0)  # LOSE
        else:
            labels.append(-1)  # NO_TRADE
    
    df['signal'] = labels
    return df

print("Creating TP/SL based labels...")
print("This may take ~20 minutes for large dataset...")

start = time.time()
train_df = create_tpsl_labels(train_df, TP_PIPS, SL_PIPS, FORWARD_BARS)
test_df = create_tpsl_labels(test_df, TP_PIPS, SL_PIPS, FORWARD_BARS)
print(f"✓ Labels created in {time.time()-start:.1f}s")

# Filter only WIN/LOSE
train_labeled = train_df[train_df['signal'] != -1].copy()
test_labeled = test_df[test_df['signal'] != -1].copy()

print(f"\n📊 Label Distribution:")
print(f"Train: {len(train_labeled):,} samples")
print(f"  WIN (TP hit): {(train_labeled['signal']==1).sum():,} ({(train_labeled['signal']==1).mean()*100:.1f}%)")
print(f"  LOSE (SL hit): {(train_labeled['signal']==0).sum():,} ({(train_labeled['signal']==0).mean()*100:.1f}%)")

print(f"\nTest: {len(test_labeled):,} samples")
print(f"  WIN (TP hit): {(test_labeled['signal']==1).sum():,} ({(test_labeled['signal']==1).mean()*100:.1f}%)")
print(f"  LOSE (SL hit): {(test_labeled['signal']==0).sum():,} ({(test_labeled['signal']==0).mean()*100:.1f}%)")

Creating TP/SL based labels...
This may take ~20 minutes for large dataset...
✓ Labels created in 1318.7s
✓ Labels created in 1318.7s

📊 Label Distribution:
Train: 601,304 samples
  WIN (TP hit): 191,049 (31.8%)
  LOSE (SL hit): 410,255 (68.2%)

Test: 121,253 samples
  WIN (TP hit): 40,811 (33.7%)
  LOSE (SL hit): 80,442 (66.3%)

📊 Label Distribution:
Train: 601,304 samples
  WIN (TP hit): 191,049 (31.8%)
  LOSE (SL hit): 410,255 (68.2%)

Test: 121,253 samples
  WIN (TP hit): 40,811 (33.7%)
  LOSE (SL hit): 80,442 (66.3%)


## 4. Prepare Data

In [5]:
# Clean NaN
train_labeled = train_labeled.dropna()
test_labeled = test_labeled.dropna()

# Define feature columns
exclude_cols = ['time', 'signal', 'open', 'high', 'low', 'close', 'volume', 'tick_volume']
feature_cols = [c for c in train_labeled.columns if c not in exclude_cols]
feature_cols = [c for c in feature_cols if train_labeled[c].dtype in ['float64', 'int64', 'float32', 'int32']]

print(f"📊 Features: {len(feature_cols)}")

X_train = train_labeled[feature_cols].values
y_train = train_labeled['signal'].values
X_test = test_labeled[feature_cols].values
y_test = test_labeled['signal'].values

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ X_train: {X_train_scaled.shape}")
print(f"✓ X_test: {X_test_scaled.shape}")
print(f"✓ y_train WIN ratio: {y_train.mean()*100:.1f}%")
print(f"✓ y_test WIN ratio: {y_test.mean()*100:.1f}%")

📊 Features: 63
✓ X_train: (601167, 63)
✓ X_test: (121231, 63)
✓ y_train WIN ratio: 31.8%
✓ y_test WIN ratio: 33.7%
✓ X_train: (601167, 63)
✓ X_test: (121231, 63)
✓ y_train WIN ratio: 31.8%
✓ y_test WIN ratio: 33.7%


## 5. Train V14 Models

In [6]:
print("="*70)
print("🚀 TRAINING V14 MODELS")
print("   V10 Features + TP/SL Labeling + Regularization")
print("="*70)

models = {}

# XGBoost - Medium Regularization
print("\n📊 Creating XGBoost models...")
models['xgb1'] = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.02,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=0.5,
    reg_lambda=2.0,
    min_child_weight=5,
    gamma=0.2,
    tree_method='hist',
    device='cuda',
    random_state=42,
    early_stopping_rounds=50,
    eval_metric='logloss'
)

models['xgb2'] = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.03,
    subsample=0.6,
    colsample_bytree=0.6,
    reg_alpha=1.0,
    reg_lambda=5.0,
    min_child_weight=10,
    gamma=0.5,
    tree_method='hist',
    device='cuda',
    random_state=43,
    early_stopping_rounds=50,
    eval_metric='logloss'
)

models['xgb3'] = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.3,
    reg_lambda=1.0,
    min_child_weight=3,
    gamma=0.1,
    tree_method='hist',
    device='cuda',
    random_state=44,
    early_stopping_rounds=50,
    eval_metric='logloss'
)

# LightGBM
print("📊 Creating LightGBM models...")
models['lgb1'] = lgb.LGBMClassifier(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.02,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=0.5,
    reg_lambda=2.0,
    min_child_samples=30,
    device='gpu',
    random_state=42,
    verbose=-1
)

models['lgb2'] = lgb.LGBMClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.03,
    subsample=0.6,
    colsample_bytree=0.6,
    reg_alpha=1.0,
    reg_lambda=5.0,
    min_child_samples=50,
    device='gpu',
    random_state=43,
    verbose=-1
)

# CatBoost
print("📊 Creating CatBoost models...")
models['cat1'] = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=0.02,
    l2_leaf_reg=5,
    min_data_in_leaf=30,
    task_type='GPU',
    random_state=42,
    verbose=0
)

models['cat2'] = CatBoostClassifier(
    iterations=500,
    depth=4,
    learning_rate=0.03,
    l2_leaf_reg=10,
    min_data_in_leaf=50,
    task_type='GPU',
    random_state=43,
    verbose=0
)

print(f"\n✓ Created {len(models)} models")

🚀 TRAINING V14 MODELS
   V10 Features + TP/SL Labeling + Regularization

📊 Creating XGBoost models...
📊 Creating LightGBM models...
📊 Creating CatBoost models...

✓ Created 7 models


In [7]:
# Train models
trained_models = {}
accuracies = {}

for name, model in models.items():
    print(f"\n🔧 Training {name}...")
    start = time.time()
    
    try:
        if 'xgb' in name:
            model.fit(
                X_train_scaled, y_train,
                eval_set=[(X_test_scaled, y_test)],
                verbose=False
            )
        else:
            model.fit(X_train_scaled, y_train)
        
        train_time = time.time() - start
        
        # Evaluate
        train_pred = model.predict(X_train_scaled)
        test_pred = model.predict(X_test_scaled)
        
        train_acc = accuracy_score(y_train, train_pred)
        test_acc = accuracy_score(y_test, test_pred)
        gap = train_acc - test_acc
        
        trained_models[name] = model
        accuracies[name] = test_acc
        
        overfit = "⚠️" if gap > 0.05 else "✓"
        print(f"   Train: {train_acc*100:.1f}%, Test: {test_acc*100:.1f}%, Gap: {gap*100:+.1f}% {overfit}")
        print(f"   Time: {train_time:.1f}s")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")

print(f"\n✓ Trained {len(trained_models)} models")


🔧 Training xgb1...
   Train: 68.3%, Test: 66.3%, Gap: +1.9% ✓
   Time: 4.1s

🔧 Training xgb2...
   Train: 68.3%, Test: 66.3%, Gap: +1.9% ✓
   Time: 4.1s

🔧 Training xgb2...
   Train: 68.2%, Test: 66.3%, Gap: +1.9% ✓
   Time: 2.2s

🔧 Training xgb3...
   Train: 68.2%, Test: 66.3%, Gap: +1.9% ✓
   Time: 2.2s

🔧 Training xgb3...
   Train: 68.3%, Test: 66.3%, Gap: +1.9% ✓
   Time: 3.5s

🔧 Training lgb1...
   Train: 68.3%, Test: 66.3%, Gap: +1.9% ✓
   Time: 3.5s

🔧 Training lgb1...
   Train: 68.5%, Test: 66.4%, Gap: +2.1% ✓
   Time: 10.1s

🔧 Training lgb2...
   Train: 68.5%, Test: 66.4%, Gap: +2.1% ✓
   Time: 10.1s

🔧 Training lgb2...
   Train: 68.4%, Test: 66.4%, Gap: +2.0% ✓
   Time: 4.1s

🔧 Training cat1...
   Train: 68.4%, Test: 66.4%, Gap: +2.0% ✓
   Time: 4.1s

🔧 Training cat1...
   Train: 68.3%, Test: 66.3%, Gap: +1.9% ✓
   Time: 7.1s

🔧 Training cat2...
   Train: 68.3%, Test: 66.3%, Gap: +1.9% ✓
   Time: 7.1s

🔧 Training cat2...
   Train: 68.3%, Test: 66.3%, Gap: +1.9% ✓
   Time: 3.

## 6. Ensemble & Evaluation

In [8]:
print("="*70)
print("📊 ENSEMBLE EVALUATION")
print("="*70)

# Calculate weights based on accuracy
total_acc = sum(accuracies.values())
weights = {name: acc/total_acc for name, acc in accuracies.items()}

print("\nModel Weights:")
for name, w in sorted(weights.items(), key=lambda x: -x[1]):
    print(f"  {name}: {w:.3f}")

# Get predictions
proba_dict = {}
preds_dict = {}
for name, model in trained_models.items():
    proba_dict[name] = model.predict_proba(X_test_scaled)
    preds_dict[name] = model.predict(X_test_scaled)

# Weighted ensemble probability
ensemble_proba = np.zeros((len(X_test_scaled), 2))
for name, proba in proba_dict.items():
    ensemble_proba += weights[name] * proba

# WIN probability
win_prob = ensemble_proba[:, 1] * 100

# Agreement bonus (V10 style)
all_preds = np.array([preds_dict[name] for name in trained_models.keys()])
win_votes = np.sum(all_preds == 1, axis=0)

all_agree = win_votes == len(trained_models)
strong_agree = win_votes >= (len(trained_models) - 1)
majority = win_votes >= (len(trained_models) // 2 + 1)

confidence = win_prob.copy()
confidence[all_agree] = np.minimum(confidence[all_agree] + 7, 100)  # +7 for all agree
confidence[strong_agree & ~all_agree] = np.minimum(confidence[strong_agree & ~all_agree] + 4, 100)
confidence[majority & ~strong_agree] = np.minimum(confidence[majority & ~strong_agree] + 2, 100)

print(f"\nConfidence Distribution:")
print(f"  >= 90%: {(confidence >= 90).sum():,}")
print(f"  >= 80%: {(confidence >= 80).sum():,}")
print(f"  >= 70%: {(confidence >= 70).sum():,}")
print(f"  >= 60%: {(confidence >= 60).sum():,}")

📊 ENSEMBLE EVALUATION

Model Weights:
  lgb2: 0.143
  lgb1: 0.143
  xgb1: 0.143
  xgb3: 0.143
  xgb2: 0.143
  cat1: 0.143
  cat2: 0.143

Confidence Distribution:
  >= 90%: 0
  >= 80%: 0
  >= 70%: 0
  >= 60%: 0

Confidence Distribution:
  >= 90%: 0
  >= 80%: 0
  >= 70%: 0
  >= 60%: 0


In [10]:
# Results by threshold - starting from lower values
print("\n📈 V14 Results by Threshold:")
print("-"*80)
print(f"{'Threshold':>10} | {'Signals':>10} | {'Wins':>10} | {'Win Rate':>10} | {'PF':>10}")
print("-"*80)

v14_results = {}
for threshold in [30, 35, 40, 45, 50, 55, 60]:
    mask = confidence >= threshold
    if mask.sum() > 0:
        signals = mask.sum()
        wins = y_test[mask].sum()
        losses = signals - wins
        win_rate = wins / signals * 100
        pf = (wins * TP_PIPS) / max(1, losses * SL_PIPS)
        
        v14_results[threshold] = {
            'signals': signals,
            'wins': wins,
            'losses': losses,
            'win_rate': win_rate,
            'pf': pf
        }
        
        marker = "🎯" if win_rate >= 40 else ""
        print(f"{threshold:>8}%+ | {signals:>10,} | {wins:>10,} | {win_rate:>9.1f}% | {pf:>9.2f} {marker}")
    else:
        print(f"{threshold:>8}%+ | {'0':>10} | {'-':>10} | {'-':>10} | {'-':>10}")

# Also show raw probability distribution
print("\n📊 Raw WIN Probability Distribution:")
print(f"  >= 50%: {(win_prob >= 50).sum():,}")
print(f"  >= 45%: {(win_prob >= 45).sum():,}")
print(f"  >= 40%: {(win_prob >= 40).sum():,}")
print(f"  >= 35%: {(win_prob >= 35).sum():,}")
print(f"  >= 30%: {(win_prob >= 30).sum():,}")


📈 V14 Results by Threshold:
--------------------------------------------------------------------------------
 Threshold |    Signals |       Wins |   Win Rate |         PF
--------------------------------------------------------------------------------
      30%+ |     84,598 |     31,403 |      37.1% |      0.89 
      35%+ |     46,313 |     18,571 |      40.1% |      1.00 🎯
      40%+ |      5,330 |      2,290 |      43.0% |      1.13 🎯
      45%+ |        161 |        102 |      63.4% |      2.59 🎯
      50%+ |         27 |         14 |      51.9% |      1.62 🎯
      55%+ |          0 |          - |          - |          -
      60%+ |          0 |          - |          - |          -

📊 Raw WIN Probability Distribution:
  >= 50%: 23
  >= 45%: 161
  >= 40%: 5,330
  >= 35%: 46,313
  >= 30%: 84,598


## 7. Overfit Check

In [12]:
# Check train vs test
print("="*70)
print("🔍 OVERFIT CHECK: Train vs Test")
print("="*70)

# Get train predictions
train_proba_dict = {}
train_preds_dict = {}
for name, model in trained_models.items():
    train_proba_dict[name] = model.predict_proba(X_train_scaled)
    train_preds_dict[name] = model.predict(X_train_scaled)

train_ensemble = np.zeros((len(X_train_scaled), 2))
for name, proba in train_proba_dict.items():
    train_ensemble += weights[name] * proba

train_win_prob = train_ensemble[:, 1] * 100

# Agreement
train_all_preds = np.array([train_preds_dict[name] for name in trained_models.keys()])
train_votes = np.sum(train_all_preds == 1, axis=0)
train_all_agree = train_votes == len(trained_models)
train_strong = train_votes >= (len(trained_models) - 1)
train_majority = train_votes >= (len(trained_models) // 2 + 1)

train_conf = train_win_prob.copy()
train_conf[train_all_agree] = np.minimum(train_conf[train_all_agree] + 7, 100)
train_conf[train_strong & ~train_all_agree] = np.minimum(train_conf[train_strong & ~train_all_agree] + 4, 100)
train_conf[train_majority & ~train_strong] = np.minimum(train_conf[train_majority & ~train_strong] + 2, 100)

print(f"\n{'Threshold':>10} | {'Train Sig':>10} | {'Train WR':>10} | {'Test Sig':>10} | {'Test WR':>10} | {'Gap':>8}")
print("-"*80)

for threshold in [30, 35, 40, 45, 50]:
    train_mask = train_conf >= threshold
    test_mask = confidence >= threshold
    
    train_sig = train_mask.sum()
    test_sig = test_mask.sum()
    
    train_wr = y_train[train_mask].mean() * 100 if train_sig > 0 else 0
    test_wr = y_test[test_mask].mean() * 100 if test_sig > 0 else 0
    
    gap = train_wr - test_wr
    status = "✓" if abs(gap) < 5 else ("⚡" if abs(gap) < 10 else "⚠️")
    
    print(f"{threshold:>8}%+ | {train_sig:>10,} | {train_wr:>9.1f}% | {test_sig:>10,} | {test_wr:>9.1f}% | {gap:>+7.1f}% {status}")

🔍 OVERFIT CHECK: Train vs Test

 Threshold |  Train Sig |   Train WR |   Test Sig |    Test WR |      Gap
--------------------------------------------------------------------------------
      30%+ |    392,873 |      38.2% |     84,598 |      37.1% |    +1.1% ✓
      35%+ |    212,981 |      43.6% |     46,313 |      40.1% |    +3.5% ✓
      40%+ |     37,019 |      57.8% |      5,330 |      43.0% |   +14.8% ⚠️
      45%+ |      1,699 |      82.0% |        161 |      63.4% |   +18.6% ⚠️
      50%+ |        231 |      97.4% |         27 |      51.9% |   +45.6% ⚠️

 Threshold |  Train Sig |   Train WR |   Test Sig |    Test WR |      Gap
--------------------------------------------------------------------------------
      30%+ |    392,873 |      38.2% |     84,598 |      37.1% |    +1.1% ✓
      35%+ |    212,981 |      43.6% |     46,313 |      40.1% |    +3.5% ✓
      40%+ |     37,019 |      57.8% |      5,330 |      43.0% |   +14.8% ⚠️
      45%+ |      1,699 |      82.0% |       

## 8. Save V14 Models

In [13]:
print("="*70)
print("💾 SAVING V14 MODELS")
print("="*70)

# Save models
for name, model in trained_models.items():
    joblib.dump(model, MODEL_DIR / f'{name}_v14.joblib')
    print(f"✓ {name}_v14.joblib")

# Save scaler
joblib.dump(scaler, MODEL_DIR / 'scaler_v14.joblib')
print(f"✓ scaler_v14.joblib")

# Save features
joblib.dump(feature_cols, MODEL_DIR / 'feature_cols_v14.joblib')
print(f"✓ feature_cols_v14.joblib")

# Save weights
joblib.dump(weights, MODEL_DIR / 'weights_v14.joblib')
print(f"✓ weights_v14.joblib")

# Save config
config = {
    'version': 'v14',
    'description': 'V10 Features + TP/SL Labeling',
    'labeling': 'TP/SL Based',
    'tp_pips': TP_PIPS,
    'sl_pips': SL_PIPS,
    'forward_bars': FORWARD_BARS,
    'models': list(trained_models.keys()),
    'features': len(feature_cols),
    'accuracies': accuracies,
    'results': v14_results
}
joblib.dump(config, MODEL_DIR / 'config_v14.joblib')
print(f"✓ config_v14.joblib")

print(f"\n✅ All V14 models saved to: {MODEL_DIR}")

💾 SAVING V14 MODELS
✓ xgb1_v14.joblib
✓ xgb2_v14.joblib
✓ xgb3_v14.joblib
✓ lgb1_v14.joblib
✓ lgb2_v14.joblib
✓ cat1_v14.joblib
✓ cat2_v14.joblib
✓ scaler_v14.joblib
✓ feature_cols_v14.joblib
✓ weights_v14.joblib
✓ config_v14.joblib

✅ All V14 models saved to: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v14


## 9. Final Summary

In [14]:
print("="*80)
print("🎯 V14 FINAL SUMMARY")
print("="*80)

# Find best threshold
valid_results = {k: v for k, v in v14_results.items() if v['signals'] >= 100}
if valid_results:
    best_th = max(valid_results.keys(), key=lambda x: valid_results[x]['pf'])
    best = v14_results[best_th]
else:
    best_th = min(v14_results.keys())
    best = v14_results[best_th]

print(f"""
📊 TRAINING SETUP:
   Features: V10-style ({len(feature_cols)} features)
   Labeling: TP/SL Based (TP={TP_PIPS} pips, SL={SL_PIPS} pips)
   Models: {len(trained_models)} ({', '.join(trained_models.keys())})
   Regularization: Medium (V10 style)

📈 BEST RESULT (Threshold {best_th}%+):
   Signals: {best['signals']:,}
   Wins: {best['wins']:,}
   Losses: {best['losses']:,}
   Win Rate: {best['win_rate']:.1f}%
   Profit Factor: {best['pf']:.2f}

💰 EXPECTED PROFIT (trades @ $1/pip):
   Gross Profit: ${int(best['wins'] * TP_PIPS):,}
   Gross Loss: ${int(best['losses'] * SL_PIPS):,}
   Net Profit: ${int(best['wins'] * TP_PIPS - best['losses'] * SL_PIPS):,}
""")

print("="*80)
print("✅ V14 TRAINING COMPLETE!")
print("="*80)

🎯 V14 FINAL SUMMARY

📊 TRAINING SETUP:
   Features: V10-style (63 features)
   Labeling: TP/SL Based (TP=15 pips, SL=10 pips)
   Models: 7 (xgb1, xgb2, xgb3, lgb1, lgb2, cat1, cat2)
   Regularization: Medium (V10 style)

📈 BEST RESULT (Threshold 45%+):
   Signals: 161
   Wins: 102
   Losses: 59
   Win Rate: 63.4%
   Profit Factor: 2.59

💰 EXPECTED PROFIT (trades @ $1/pip):
   Gross Profit: $1,530
   Gross Loss: $590
   Net Profit: $940

✅ V14 TRAINING COMPLETE!
